In [1]:
import pandas as pd
import plotly.graph_objects as go
from signals import *

signal_file = "./telegram/2020-21-01_TelegramDump.json"

In [ ]:
# Analyse difference between stated entry price and the actual entry price
signals = load_beta_trades(signal_file)
for signal in signals:
    df = load_ticker(signal.ticker)
    ohlc_values = df.loc[signal.datetime_str:].iterrows()
    date, first_ohlc = next(ohlc_values)
    slippage = pip_factor(signal.ticker) * abs(first_ohlc.Open - signal.entry_price)
    if slippage > 10:
        print(date, "BOOM", signal.ticker, signal.entry_price, first_ohlc.Open, slippage)
    else:
        print(date, signal.ticker, signal.entry_price, first_ohlc.Open, signal.stop_loss)
        


In [4]:
# Example plotly candlestick plot
#2019-06-10 05:30:00 EURGBP Signal Entry = 0.8896 Real Entry = 0.8894 TP = 0.9005
#2019-05-14 14:40:00 BOOM EURGBP 1.8685 0.8679 10006.0
start_dt = "2019-05-14 13:40"
end_dt = "2019-05-28 14:40"
df = get_supertrend("EURGBP", multiplier=6, length=40, sample_mins=60)
dfp = df.loc[start_dt:end_dt]
fig = go.Figure(data=[go.Candlestick(x=dfp.index,
                open=dfp['Open'],
                high=dfp['High'],
                low=dfp['Low'],
                close=dfp['Close'])])

fig.add_trace(go.Scatter(x=dfp.index, y=dfp.SuperTrend,
                    mode='lines',
                    name='lines'))

# Entry Cross
fig.add_shape(go.layout.Shape(            
                type="line",
                x0="2019-05-14 13:40",
                y0=0.8685,
                x1="2019-05-14 15:40",
                y1=0.8685,
                line=dict(
                    color="RoyalBlue",
                    width=1
                )
            )
        )
fig.add_shape(go.layout.Shape(            
                type="line",
                x0="2019-05-14 14:40",
                y0=0.8685*(1+0.0005),
                x1="2019-05-14 14:40",
                y1=0.8685*(1-0.0005),
                line=dict(
                    color="RoyalBlue",
                    width=1
                )
            )
        )

# Stop Loss
#fig.add_shape(go.layout.Shape(            
#                type="line",
#                x0=start_dt,
#                y0=0.9005,
#                x1=end_dt,
#                y1=0.9005,
#                line=dict(
#                    color="Red",
#                    width=1
#                )
#            )
#        )

fig.show()